# Importação e organização inicial dos dados

In [16]:
import pandas as pd
pd.set_option('max_colwidth', 300)

# 1 - Importação do arquivo

fonte: https://www.crowdflower.com/wp-content/uploads/2016/03/twitter-hate-speech-classifier-DFE-a845520.csv



In [17]:
data = pd.read_csv(
    "twitter-hate-speech-classifier-DFE-a845520.csv", 
    encoding='ISO-8859-1', 
    usecols=[
        'tweet_text', 
        'does_this_tweet_contain_hate_speechconfidence', 
        'does_this_tweet_contain_hate_speech:confidence',
        'does_this_tweet_contain_hate_speech'])

In [18]:
data = data.rename(columns={
    'does_this_tweet_contain_hate_speech:confidence':'confidence',
    'does_this_tweet_contain_hate_speechconfidence':'hate_speech:confidence'
})

In [19]:
data = data.reindex(columns=[
    'tweet_text', 
    'hate_speech:confidence',
    'does_this_tweet_contain_hate_speech',
    'confidence'])

In [20]:
data.head()

,tweet_text,hate_speech:confidence,does_this_tweet_contain_hate_speech,confidence
0,Warning: penny boards will make you a faggot,1.0,The tweet uses offensive language but not hate speech,0.6013
1,Fuck dykes,1.0,The tweet contains hate speech,0.7227
2,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandonernandez @bootyacid at least i dont look like jefree starr faggot,1.0,The tweet contains hate speech,0.5229
3,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkillah Is a fag"" jackie jealous"" Neeeee",1.0,The tweet contains hate speech,0.5184
4,@Zhugstubble You heard me bitch but any way I'm back th texas so wtf u talking about bitch ass nigga,1.0,The tweet uses offensive language but not hate speech,0.5185


In [21]:
data.describe()

,hate_speech:confidence,confidence
count,67.0,14509.000000
mean,1.0,0.865844
std,0.0,0.178734
min,1.0,0.333300
25%,1.0,0.668400
50%,1.0,1.000000
75%,1.0,1.000000
max,1.0,1.000000


# Labels
1. 'The tweet uses offensive language but not hate speech' = O
2. 'The tweet contains hate speech' = H
3. 'The tweet is not offensive' = N

In [22]:
labels = {
    'The tweet uses offensive language but not hate speech': 'O',
    'The tweet contains hate speech': 'H',
    'The tweet is not offensive': 'N'
}

In [23]:
data['labels'] = [labels[value[2]] for value in data.values]
data = data.drop(columns=['does_this_tweet_contain_hate_speech', 'hate_speech:confidence'])
data['length'] = [len(value[0]) for value in data.values]

In [24]:
data.describe()

,confidence,length
count,14509.000000,14509.000000
mean,0.865844,88.857606
std,0.178734,38.724537
min,0.333300,4.000000
25%,0.668400,55.000000
50%,1.000000,89.000000
75%,1.000000,125.000000
max,1.000000,469.000000


In [25]:
hate_data = pd.DataFrame.from_dict(
    [dict(zip(data.columns,v))for v in data.values if v[2]=='H']
)

In [26]:
hate_data.sort_values(by='length')

,tweet_text,confidence,labels,length
0,Fuck dykes,0.7227,H,10
1523,You nigger,0.6650,H,10
59,wtf faggot,1.0000,H,10
2382,stupid fag,1.0000,H,10
363,Coon shit!,0.6689,H,10
...,...,...,...,...
51,That moment when a faggot creeps through Twitter to find your Xbox Twitter without your name anywhere so he can copy my posts about him_ã¢Û_Ì__ã¢Û_Ì_ÊÛ_å_ÊÛ_å_ÊÛ_å_,0.6650,H,174
5,Well I thought you knew actually RT @KingHorseDick: Man why y'all didn't tell me I was a dick riding ass faggot? Y'all not real &#128557;&#128557;&#128557;&#128557;&#128557;&#128557;,0.5619,H,182
2067,This guy is the biggest faggot ever _Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡ buddy you called me a hoe and now you wanna talk to me again _Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡_Ì«ÌÐÛ¡ gtfo,0.6735,H,183
2064,Happy birthday to the biggest Faggot going @nicolemay__ have a fab day baby love you to the moon and back _Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì__Ì«Ì_,0.6754,H,190


In [27]:
graph = dict(zip(hate_data['confidence'], hate_data.length))
graph = pd.DataFrame.from_dict(graph, orient='index')

In [28]:
graph = graph.sort_index()